In [9]:
# Cell 1: Import Required Libraries
# =====================================

# Core Python libraries
import json
import os
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
import base64
from io import BytesIO

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

# HTTP requests for API calls
import requests

# Environment variables (for API keys)
from dotenv import load_dotenv

# Rich text formatting for better notebook output
from IPython.display import display, HTML, Image, Markdown

# Warnings management
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables if .env file exists
load_dotenv()

False

In [15]:
# Cell 2: Configuration and API Setup
# =====================================

# API Configuration
OPENROUTER_API_KEY = "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"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_NAME = "deepseek/deepseek-chat-v3-0324:free"  # Deepseek model via OpenRouter.
# Deepseek Free is often rate-limited due to the amount of people using it at once. Sometimes you may need to wait 1-5 minutes between requests. 

# Headers for API requests
HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost:8888",  # Required for OpenRouter
    "X-Title": "Dental Practice Valuation Q&A"  # Optional, helps with OpenRouter analytics
}

# Question Categories (for routing to appropriate handlers)
QUESTION_CATEGORIES = {
    "P&L_ANALYSIS": ["profit", "revenue", "expense", "margin", "ebitda", "income"],
    "TAX": ["tax", "deduction", "write-off", "depreciation", "amortization"],
    "VALUATION": ["multiple", "valuation", "worth", "value", "price"],
    "OPERATIONS": ["patient", "operatories", "technology", "location", "service"],
    "SDE": ["sde", "seller", "discretionary", "earnings", "addback", "adjustment"],
    "SWOT": ["strength", "weakness", "opportunity", "threat", "swot", "growth"],
    "DEBT": ["debt", "loan", "financing", "interest", "service"],
    "GENERAL": ["tell me about", "overview", "summary", "explain"]
}

# Visualization preferences
VIZ_CONFIG = {
    "figure_size": (10, 6),
    "color_scheme": ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D", "#6A994E"],
    "font_size": 12,
    "title_size": 14,
    "dpi": 100,
    "style": "seaborn-v0_8-darkgrid"
}

# Financial formatting preferences
FINANCIAL_CONFIG = {
    "currency_symbol": "$",
    "thousands_separator": ",",
    "decimal_places": 0,
    "percentage_decimal": 1
}

# System prompt for the LLM
SYSTEM_PROMPT = """You are a dental practice valuation expert AI assistant. You analyze financial data, 
operational metrics, and practice characteristics to provide insights about dental practice valuations.

Your responses should be:
1. Accurate and based solely on the provided data
2. Professional but accessible to non-financial professionals
3. Focused on actionable insights
4. Clear about calculations and methodologies used

When analyzing data:
- Always show key calculations
- Explain financial terms when first used
- Highlight important ratios and benchmarks
- Provide context for valuation multiples
"""

# Test API connection
def test_api_connection():
    """Test the OpenRouter API connection with Deepseek"""
    test_payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Respond with 'Connection successful!' if you receive this."}
        ],
        "max_tokens": 50,
        "temperature": 0.1
    }
    
    try:
        response = requests.post(
            OPENROUTER_BASE_URL,
            headers=HEADERS,
            json=test_payload
        )
        
        if response.status_code == 200:
            result = response.json()
            message = result['choices'][0]['message']['content']
            print(f"✅ API Connection Test Successful!")
            print(f"📡 Model: {MODEL_NAME}")
            print(f"💬 Response: {message}")
            return True
        else:
            print(f"❌ API Connection Failed!")
            print(f"Status Code: {response.status_code}")
            print(f"Error: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Connection Error: {str(e)}")
        return False

# Run connection test
print("=" * 50)
print("DENTAL PRACTICE VALUATION Q&A SYSTEM")
print("=" * 50)
print(f"\n🔧 Configuration loaded successfully!")
print(f"🤖 Using model: {MODEL_NAME}")
print(f"\n📡 Testing API connection...")
print("-" * 50)

connection_status = test_api_connection()

if connection_status:
    print("-" * 50)
    print("✅ System ready for use!")
else:
    print("-" * 50)
    print("⚠️  Please check your API key and connection")

DENTAL PRACTICE VALUATION Q&A SYSTEM

🔧 Configuration loaded successfully!
🤖 Using model: deepseek/deepseek-chat-v3-0324:free

📡 Testing API connection...
--------------------------------------------------
✅ API Connection Test Successful!
📡 Model: deepseek/deepseek-chat-v3-0324:free
💬 Response: Connection successful!
--------------------------------------------------
✅ System ready for use!


In [11]:
# Cell 3: Data Loading and Validation Functions
# ===============================================

class DentalPracticeData:
    """
    Main class to handle all dental practice valuation data.
    Think of this as a container that holds and processes all the practice's financial info.
    """
    
    def __init__(self, json_path: str = None, json_data: dict = None):
        """
        Initialize the class when creating a new instance.
        You can either:
        1. Pass a file path to load JSON from a file, OR
        2. Pass the JSON data directly as a dictionary
        
        Example usage:
        - From file: practice_data = DentalPracticeData(json_path="demo_clinic.json")
        - From dict: practice_data = DentalPracticeData(json_data=my_json_dict)
        """
        if json_path:
            self.data = self.load_from_file(json_path)  # Load from file
        elif json_data:
            self.data = json_data  # Use provided dictionary
        else:
            raise ValueError("Must provide either json_path or json_data")
        
        # After loading, validate the data has all required fields
        self.validate_data()
        
        # Calculate additional metrics that aren't in the raw data
        # (like margins, ratios, etc.)
        self.calculate_derived_metrics()
    
    def load_from_file(self, filepath: str) -> dict:
        """
        Loads JSON data from a file on disk.
        
        Parameters:
        - filepath: The path to the JSON file (e.g., "demo_clinic.json")
        
        Returns:
        - Dictionary containing all the JSON data
        
        What it does:
        1. Opens the file
        2. Reads the JSON content
        3. Converts it to a Python dictionary
        4. Returns the dictionary for use
        """
        try:
            with open(filepath, 'r') as f:  # Open file in read mode
                data = json.load(f)  # Parse JSON into Python dict
            print(f"✅ Successfully loaded data from {filepath}")
            return data
        except FileNotFoundError:
            print(f"❌ File not found: {filepath}")
            raise  # Re-raise the error so caller knows what happened
        except json.JSONDecodeError as e:
            print(f"❌ Invalid JSON format: {e}")
            raise
    
    def validate_data(self):
        """
        Checks that the JSON has all the fields we need for analysis.
        Think of this as a quality check before we start processing.
        
        What it checks:
        1. All main required fields are present (Revenue, EBITDA, etc.)
        2. All SDE components are included (for the SDE waterfall chart)
        3. Prints warnings if anything is missing
        
        Why this matters:
        - Prevents errors later when we try to access missing data
        - Ensures we can generate all visualizations properly
        """
        # List of fields that MUST be in the JSON
        required_fields = [
            "Clinic",               # Practice name
            "Revenue_Trailing12",   # Last 12 months revenue
            "EBITDA_Trailing12",    # Last 12 months EBITDA
            "Total_SDE",           # Seller's Discretionary Earnings total
            "SDE",                 # Breakdown of SDE components
            "Valuation_Multiples"  # Multiples for valuation calculations
        ]
        
        # Check each required field
        missing_fields = []
        for field in required_fields:
            if field not in self.data:  # If field is missing
                missing_fields.append(field)
        
        # If any required fields are missing, stop and report error
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # Check SDE components (these show how we calculated SDE)
        sde_components = [
            "Net_Profit_pre_tax",      # Base profit before taxes
            "Owner_Salary_minus_Market", # Excess owner compensation
            "Owner_Perks",             # Personal expenses paid by business
            "Interest",                # Interest (will be restructured)
            "Depreciation",            # Non-cash expense
            "Amortization",            # Non-cash expense
            "Non_Recurring"            # One-time expenses
        ]
        
        # Check if all SDE components are present
        if "SDE" in self.data:
            missing_sde = [c for c in sde_components if c not in self.data["SDE"]]
            if missing_sde:
                print(f"⚠️  Warning: Missing SDE components: {missing_sde}")
        
        print("✅ Data validation successful!")
    
    def calculate_derived_metrics(self):
        """
        Calculates additional financial metrics from the base data.
        These are metrics that aren't directly in the JSON but are useful for analysis.
        
        What it calculates:
        1. EBITDA Margin % = (EBITDA / Revenue) * 100
        2. SDE Margin % = (SDE / Revenue) * 100  
        3. Implied practice value using EBITDA multiple
        4. Implied practice value using Revenue multiple
        5. Debt Service Coverage Ratio (DSCR) = EBITDA / Annual Debt Payment
        6. After-tax SDE and cash flow
        
        These metrics help answer questions like:
        - "How profitable is the practice?"
        - "What's the practice worth?"
        - "Can it support the debt payments?"
        """
        # Create a dictionary to store calculated metrics
        self.metrics = {}
        
        # --- PROFITABILITY METRICS ---
        revenue = self.data.get("Revenue_Trailing12", 0)  # Get revenue, default to 0 if missing
        
        if revenue > 0:  # Only calculate if we have revenue (avoid division by zero)
            # EBITDA Margin: What % of revenue becomes EBITDA?
            self.metrics["EBITDA_Margin"] = (self.data.get("EBITDA_Trailing12", 0) / revenue) * 100
            
            # SDE Margin: What % of revenue becomes SDE?
            self.metrics["SDE_Margin"] = (self.data.get("Total_SDE", 0) / revenue) * 100
        
        # --- VALUATION METRICS ---
        if "Valuation_Multiples" in self.data:
            multiples = self.data["Valuation_Multiples"]
            
            # Practice value = EBITDA × EBITDA multiple
            # Example: $545,301 × 4.5 = $2,453,854
            self.metrics["Implied_Value_EBITDA"] = (
                self.data.get("EBITDA_Trailing12", 0) * 
                multiples.get("EBITDA_multiple", 0)
            )
            
            # Practice value = Revenue × Revenue multiple  
            # Example: $1,546,696 × 0.8 = $1,237,356
            self.metrics["Implied_Value_Revenue"] = (
                revenue * multiples.get("Revenue_multiple", 0)
            )
        
        # --- DEBT SERVICE METRICS ---
        # DSCR shows if practice generates enough cash to pay debts
        # DSCR > 1.25 is typically considered healthy
        if "Debt_Service_Annual" in self.data and self.data["Debt_Service_Annual"] > 0:
            self.metrics["DSCR"] = (
                self.data.get("EBITDA_Trailing12", 0) / 
                self.data["Debt_Service_Annual"]
            )
        
        # --- AFTER-TAX CALCULATIONS ---
        # Shows what buyer keeps after paying taxes
        if "Buyer_Tax_Rate" in self.data:
            tax_rate = self.data["Buyer_Tax_Rate"]  # e.g., 0.25 = 25%
            
            # After-tax SDE = SDE × (1 - tax rate)
            # Example: $460,000 × (1 - 0.25) = $345,000
            self.metrics["After_Tax_SDE"] = (
                self.data.get("Total_SDE", 0) * (1 - tax_rate)
            )
            
            # Cash flow after taxes AND debt payments
            # This is what the buyer actually keeps
            if "Debt_Service_Annual" in self.data:
                self.metrics["After_Tax_Cash_Flow"] = (
                    self.metrics["After_Tax_SDE"] - 
                    self.data["Debt_Service_Annual"]
                )
    
    def get_summary(self) -> pd.DataFrame:
        """
        Creates a clean summary table of all key metrics.
        
        Returns:
        - A pandas DataFrame (table) with three columns:
          1. Metric name
          2. Formatted value (with $ signs and commas)
          3. Category (Revenue, Profitability, Valuation, etc.)
        
        This table is perfect for:
        - Quick overview presentations
        - Exporting to Excel
        - Showing to clients
        """
        # Initialize empty lists to build our table
        summary_data = {
            "Metric": [],      # Name of the metric
            "Value": [],       # Formatted value
            "Category": []     # Type of metric
        }
        
        # --- Add Revenue metrics ---
        summary_data["Metric"].append("Revenue (TTM)")  # TTM = Trailing Twelve Months
        summary_data["Value"].append(f"${self.data.get('Revenue_Trailing12', 0):,.0f}")  # Format with commas
        summary_data["Category"].append("Revenue")
        
        # --- Add Profitability metrics ---
        summary_data["Metric"].append("EBITDA (TTM)")
        summary_data["Value"].append(f"${self.data.get('EBITDA_Trailing12', 0):,.0f}")
        summary_data["Category"].append("Profitability")
        
        summary_data["Metric"].append("Total SDE")
        summary_data["Value"].append(f"${self.data.get('Total_SDE', 0):,.0f}")
        summary_data["Category"].append("Profitability")
        
        # --- Add Margin percentages (if calculated) ---
        if "EBITDA_Margin" in self.metrics:
            summary_data["Metric"].append("EBITDA Margin")
            summary_data["Value"].append(f"{self.metrics['EBITDA_Margin']:.1f}%")  # 1 decimal place
            summary_data["Category"].append("Profitability")
        
        if "SDE_Margin" in self.metrics:
            summary_data["Metric"].append("SDE Margin")
            summary_data["Value"].append(f"{self.metrics['SDE_Margin']:.1f}%")
            summary_data["Category"].append("Profitability")
        
        # --- Add Valuation estimates ---
        if "Implied_Value_EBITDA" in self.metrics:
            summary_data["Metric"].append("Implied Value (EBITDA)")
            summary_data["Value"].append(f"${self.metrics['Implied_Value_EBITDA']:,.0f}")
            summary_data["Category"].append("Valuation")
        
        if "Implied_Value_Revenue" in self.metrics:
            summary_data["Metric"].append("Implied Value (Revenue)")
            summary_data["Value"].append(f"${self.metrics['Implied_Value_Revenue']:,.0f}")
            summary_data["Category"].append("Valuation")
        
        # --- Add Debt metrics ---
        if "DSCR" in self.metrics:
            summary_data["Metric"].append("Debt Service Coverage")
            summary_data["Value"].append(f"{self.metrics['DSCR']:.2f}x")  # Show as "2.45x"
            summary_data["Category"].append("Debt")
        
        # Convert to pandas DataFrame for nice display
        return pd.DataFrame(summary_data)
    
    def display_practice_overview(self):
        """
        Prints a nicely formatted overview of the practice.
        This is what you'd show a client for a quick summary.
        
        Displays:
        1. Practice name
        2. Location and operational details
        3. Key financial metrics
        4. SWOT highlights (main strength and opportunity)
        
        This gives a complete picture in a easy-to-read format.
        """
        # Print header with practice name
        print("\n" + "="*60)
        print(f"PRACTICE: {self.data.get('Clinic', 'Unknown')}")
        print("="*60)
        
        # --- Display practice details (if available) ---
        if "Practice_Overview" in self.data:
            overview = self.data["Practice_Overview"]
            print("\n📍 PRACTICE DETAILS:")
            print(f"   Location: {overview.get('Location', 'N/A')}")
            print(f"   Patient Base: {overview.get('Patient_Base', 'N/A')}")
            print(f"   Operatories: {overview.get('Operatories', 'N/A')}")
            
            # List all technology (join array into comma-separated string)
            if "Technology" in overview:
                print(f"   Technology: {', '.join(overview['Technology'])}")
        
        # --- Display financial metrics ---
        print("\n💰 KEY FINANCIAL METRICS:")
        summary_df = self.get_summary()  # Get the summary table
        for _, row in summary_df.iterrows():  # Loop through each row
            print(f"   {row['Metric']}: {row['Value']}")
        
        # --- Display SWOT highlights (just the first/most important) ---
        if "SWOT" in self.data:
            swot = self.data["SWOT"]
            print("\n📊 SWOT HIGHLIGHTS:")
            
            # Show first strength (most important)
            if "Strengths" in swot and swot["Strengths"]:
                print(f"   Key Strength: {swot['Strengths'][0]}")
            
            # Show first opportunity (highest potential)
            if "Opportunities" in swot and swot["Opportunities"]:
                print(f"   Top Opportunity: {swot['Opportunities'][0]}")

# ===============================================
# TEST THE CLASS WITH DEMO DATA
# ===============================================

print("Testing data loading with demo_clinic.json...")
print("-" * 60)

# Try to load the demo data file
try:
    # Create an instance of our class with the demo file
    practice_data = DentalPracticeData(json_path="demo_clinic.json")
    
    # Display the overview
    practice_data.display_practice_overview()
    
    # Show the summary table
    print("\n📊 Summary DataFrame:")
    display(practice_data.get_summary())
    
    print("\n✅ Data loading module ready!")
    
except FileNotFoundError:
    # If file doesn't exist, create it from the sample data
    print("\n⚠️  demo_clinic.json not found. Creating from provided data...")
    
    # This is the complete demo data structure
    demo_data = {
        "Clinic": "Demo Dental NYC",
        "Valuation_Date": "2025-07-22",
        "Revenue_Trailing12": 1546696,
        "EBITDA_Trailing12": 545301,
        "Total_SDE": 460000,
        "SDE": {
            "Net_Profit_pre_tax": 350000,
            "Owner_Salary_minus_Market": 60000,
            "Owner_Perks": 18000,
            "Interest": 12000,
            "Depreciation": 9000,
            "Amortization": 4000,
            "Non_Recurring": 7000
        },
        "Addback_Justifications": {
            "Net_Profit_pre_tax": "Base accounting profit before income tax adjustments.",
            "Owner_Salary_minus_Market": "Portion of DDS salary above fair‑market compensation.",
            "Owner_Perks": "Clinic‑paid personal auto, phone, and family health insurance.",
            "Interest": "Financing cost—assumed refinanced by buyer, thus added back.",
            "Depreciation": "Non‑cash charge; added back to reflect cash flow.",
            "Amortization": "Non‑cash; pertains to prior goodwill amortization.",
            "Non_Recurring": "One‑time legal settlement expense in FY 2024."
        },
        "Valuation_Multiples": {"EBITDA_multiple": 4.5, "Revenue_multiple": 0.8},
        "Debt_Service_Annual": 168008,
        "Buyer_Tax_Rate": 0.25,
        "Practice_Overview": {
            "Location": "Midtown Manhattan, NY",
            "Patient_Base": "95% Fee‑For‑Service, 2100 active patients",
            "Operatories": 6,
            "Technology": ["Digital X‑ray", "Itero Scanner", "CBCT"]
        },
        "SWOT": {
            "Strengths": ["High FFS revenue", "Prime Manhattan location", "Robust hygiene program"],
            "Weaknesses": ["Limited Saturday hours"],
            "Opportunities": ["Add implant services", "Expand evening hours"],
            "Threats": ["Rising rent costs"]
        }
    }
    
    # Save the demo data to a file
    with open("demo_clinic.json", "w") as f:
        json.dump(demo_data, f, indent=2)  # indent=2 makes it readable
    
    print("✅ Created demo_clinic.json")
    
    # Now load it using our class
    practice_data = DentalPracticeData(json_path="demo_clinic.json")
    practice_data.display_practice_overview()
    
    print("\n📊 Summary DataFrame:")
    display(practice_data.get_summary())
    
    print("\n✅ Data loading module ready!")

Testing data loading with demo_clinic.json...
------------------------------------------------------------
✅ Successfully loaded data from demo_clinic.json
✅ Data validation successful!

PRACTICE: Demo Dental NYC

📍 PRACTICE DETAILS:
   Location: Midtown Manhattan, NY
   Patient Base: 95% Fee‑For‑Service, 2100 active patients
   Operatories: 6
   Technology: Digital X‑ray, Itero Scanner, CBCT

💰 KEY FINANCIAL METRICS:
   Revenue (TTM): $1,546,696
   EBITDA (TTM): $545,301
   Total SDE: $460,000
   EBITDA Margin: 35.3%
   SDE Margin: 29.7%
   Implied Value (EBITDA): $2,453,854
   Implied Value (Revenue): $1,237,357
   Debt Service Coverage: 3.25x

📊 SWOT HIGHLIGHTS:
   Key Strength: High FFS revenue
   Top Opportunity: Add implant services

📊 Summary DataFrame:


,Metric,Value,Category
0,Revenue (TTM),"$1,546,696",Revenue
1,EBITDA (TTM),"$545,301",Profitability
2,Total SDE,"$460,000",Profitability
3,EBITDA Margin,35.3%,Profitability
4,SDE Margin,29.7%,Profitability
5,Implied Value (EBITDA),"$2,453,854",Valuation
6,Implied Value (Revenue),"$1,237,357",Valuation
7,Debt Service Coverage,3.25x,Debt



✅ Data loading module ready!


In [13]:
# Cell 4: LLM Interaction Functions
# ===================================

def categorize_question(question: str) -> str:
    """
    Determines what type of question the user is asking.
    This helps us route to the right type of analysis and visualization.
    
    Parameters:
    - question: The user's question as a string
    
    Returns:
    - Category name (e.g., "SDE", "VALUATION", "TAX", etc.)
    
    How it works:
    1. Converts question to lowercase for matching
    2. Checks for keywords associated with each category
    3. Returns the first matching category
    4. Defaults to "GENERAL" if no specific match
    """
    question_lower = question.lower()
    
    # Check each category's keywords
    for category, keywords in QUESTION_CATEGORIES.items():
        for keyword in keywords:
            if keyword in question_lower:
                return category
    
    return "GENERAL"  # Default category

def create_context_prompt(practice_data: DentalPracticeData, question: str, category: str) -> str:
    """
    Creates a detailed prompt for the LLM with all relevant practice data.
    This gives DeepSeek all the context it needs to answer accurately.
    
    Parameters:
    - practice_data: The DentalPracticeData object with all financial info
    - question: The user's question
    - category: The question category (from categorize_question)
    
    Returns:
    - A complete prompt string for the LLM
    
    The prompt includes:
    1. The full practice data as JSON
    2. Calculated metrics (margins, ratios, etc.)
    3. Specific instructions based on question category
    4. The user's actual question
    """
    
    # Convert all practice data to a formatted string
    data_context = json.dumps(practice_data.data, indent=2)
    
    # Add calculated metrics to context
    metrics_context = "\nCALCULATED METRICS:\n"
    for metric_name, metric_value in practice_data.metrics.items():
        if isinstance(metric_value, float):
            if "margin" in metric_name.lower() or "rate" in metric_name.lower():
                metrics_context += f"- {metric_name}: {metric_value:.1f}%\n"
            else:
                metrics_context += f"- {metric_name}: ${metric_value:,.0f}\n"
        else:
            metrics_context += f"- {metric_name}: {metric_value}\n"
    
    # Create category-specific instructions
    category_instructions = {
        "SDE": """
        Focus on Seller's Discretionary Earnings (SDE) components and calculations.
        Explain each addback and its justification. Show the build-up from Net Profit to SDE.
        Highlight which components are cash vs non-cash adjustments.
        """,
        
        "VALUATION": """
        Explain the valuation multiples and how they're applied.
        Compare EBITDA-based vs Revenue-based valuations.
        Discuss what these multiples mean in the dental industry context.
        Mention typical ranges for dental practices if relevant.
        """,
        
        "TAX": """
        Focus on tax implications and after-tax returns.
        Explain how depreciation and amortization affect taxes.
        Calculate after-tax cash flows for the buyer.
        Consider the impact of the buyer's tax rate.
        """,
        
        "P&L_ANALYSIS": """
        Analyze profitability metrics and margins.
        Compare to industry benchmarks where relevant.
        Identify areas of strong performance or concern.
        Focus on EBITDA and operating margins.
        """,
        
        "SWOT": """
        Provide detailed analysis of Strengths, Weaknesses, Opportunities, and Threats.
        Explain how each factor impacts practice value.
        Suggest actionable recommendations based on the SWOT.
        """,
        
        "OPERATIONS": """
        Discuss operational aspects like patient base, technology, and capacity.
        Explain how these factors affect practice value and efficiency.
        Consider growth potential based on current operations.
        """,
        
        "DEBT": """
        Analyze debt service coverage and financing implications.
        Calculate how debt payments affect buyer cash flow.
        Discuss the sustainability of current debt levels.
        """,
        
        "GENERAL": """
        Provide a comprehensive answer based on all available data.
        Focus on the most relevant metrics for the question asked.
        Be specific and use actual numbers from the practice data.
        """
    }
    
    # Build the complete prompt
    prompt = f"""You are analyzing a dental practice valuation. Here is the complete practice data:

PRACTICE DATA:
{data_context}

{metrics_context}

QUESTION CATEGORY: {category}

SPECIFIC INSTRUCTIONS:
{category_instructions.get(category, category_instructions["GENERAL"])}

USER QUESTION: {question}

Please provide a detailed, professional response that:
1. Directly answers the question using specific data from this practice
2. Shows relevant calculations where appropriate
3. Provides context and interpretation of the numbers
4. Suggests what visualizations would be helpful (we'll generate these separately)

Be specific with numbers and avoid generic responses. Format your response clearly with sections if needed."""
    
    return prompt

def call_llm(prompt: str, temperature: float = 0.3) -> str:
    """
    Sends the prompt to DeepSeek and gets the response.
    
    Parameters:
    - prompt: The complete prompt to send
    - temperature: Controls randomness (0=deterministic, 1=creative)
                  Default 0.3 for factual financial analysis
    
    Returns:
    - The LLM's response as a string
    
    Error handling:
    - Catches API errors and returns helpful error messages
    - Logs the error for debugging
    """
    
    # Prepare the API request
    payload = {
        "model": MODEL_NAME,  # deepseek/deepseek-chat-v3-0324:free
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},  # Set the LLM's role
            {"role": "user", "content": prompt}  # The actual question/prompt
        ],
        "max_tokens": 1500,  # Limit response length
        "temperature": temperature  # Control randomness
    }
    
    try:
        # Make the API call
        print("🤖 Calling DeepSeek API...")
        response = requests.post(
            OPENROUTER_BASE_URL,
            headers=HEADERS,
            json=payload,
            timeout=30  # 30 second timeout
        )
        
        # Check if request was successful
        if response.status_code == 200:
            result = response.json()
            # Extract the actual message from the response
            llm_response = result['choices'][0]['message']['content']
            print("✅ Response received!")
            return llm_response
        else:
            # API returned an error
            error_msg = f"API Error: {response.status_code} - {response.text}"
            print(f"❌ {error_msg}")
            return f"Sorry, I encountered an error: {error_msg}"
            
    except requests.exceptions.Timeout:
        return "Sorry, the request timed out. Please try again."
    except requests.exceptions.RequestException as e:
        return f"Sorry, I encountered a connection error: {str(e)}"
    except Exception as e:
        return f"Sorry, an unexpected error occurred: {str(e)}"

def get_visualization_recommendations(question: str, category: str, practice_data: DentalPracticeData) -> list:
    """
    Determines what visualizations would be helpful for the question.
    Returns a list of recommended chart types with specifications.
    
    Parameters:
    - question: The user's question
    - category: Question category
    - practice_data: The practice data object
    
    Returns:
    - List of dictionaries, each describing a visualization to create
    
    This function acts as a "visualization router" that decides
    what charts would best support the answer.
    """
    
    recommendations = []
    
    # SDE questions → Waterfall chart showing build-up
    if category == "SDE":
        recommendations.append({
            "type": "waterfall",
            "title": "SDE Build-up Analysis",
            "description": "Shows how we get from Net Profit to SDE",
            "data": "SDE components"
        })
    
    # Valuation questions → Comparison bar chart
    elif category == "VALUATION":
        recommendations.append({
            "type": "bar_comparison",
            "title": "Valuation Methods Comparison",
            "description": "EBITDA-based vs Revenue-based valuations",
            "data": "Valuation multiples"
        })
    
    # P&L Analysis → Margin charts
    elif category == "P&L_ANALYSIS":
        recommendations.append({
            "type": "bar_chart",
            "title": "Profitability Margins",
            "description": "Key margin percentages",
            "data": "Margins"
        })
        recommendations.append({
            "type": "pie_chart",
            "title": "Expense Breakdown",
            "description": "Where the money goes",
            "data": "Expense categories"
        })
    
    # SWOT → Matrix visualization
    elif category == "SWOT":
        recommendations.append({
            "type": "swot_matrix",
            "title": "SWOT Analysis Matrix",
            "description": "Strategic position overview",
            "data": "SWOT"
        })
    
    # Tax questions → After-tax cash flow
    elif category == "TAX":
        recommendations.append({
            "type": "funnel_chart",
            "title": "After-Tax Cash Flow",
            "description": "From SDE to take-home cash",
            "data": "Tax calculations"
        })
    
    # Debt questions → Coverage ratio gauge
    elif category == "DEBT":
        recommendations.append({
            "type": "gauge_chart",
            "title": "Debt Service Coverage Ratio",
            "description": "Ability to service debt",
            "data": "DSCR"
        })
    
    # General questions → Dashboard overview
    else:
        recommendations.append({
            "type": "dashboard",
            "title": "Practice Overview Dashboard",
            "description": "Key metrics at a glance",
            "data": "Summary metrics"
        })
    
    return recommendations

def answer_question(practice_data: DentalPracticeData, question: str) -> dict:
    """
    Main function that orchestrates the entire Q&A process.
    This is what you'll call to get an answer to any question.
    
    Parameters:
    - practice_data: The loaded practice data
    - question: The user's question
    
    Returns:
    - Dictionary containing:
      - 'question': The original question
      - 'category': Detected category
      - 'answer': The LLM's response
      - 'visualizations': Recommended charts to create
      - 'metrics_used': Key metrics referenced in the answer
    
    Process:
    1. Categorize the question
    2. Build context-aware prompt
    3. Get LLM response
    4. Determine visualizations needed
    5. Package everything together
    """
    
    print(f"\n{'='*60}")
    print(f"📝 QUESTION: {question}")
    print(f"{'='*60}")
    
    # Step 1: Categorize the question
    category = categorize_question(question)
    print(f"📊 Category detected: {category}")
    
    # Step 2: Create the prompt with all context
    prompt = create_context_prompt(practice_data, question, category)
    
    # Step 3: Get the LLM's answer
    answer = call_llm(prompt)
    
    # Step 4: Get visualization recommendations
    viz_recommendations = get_visualization_recommendations(question, category, practice_data)
    
    # Step 5: Extract key metrics mentioned (for highlighting)
    metrics_used = []
    if "EBITDA" in answer:
        metrics_used.append(f"EBITDA: ${practice_data.data.get('EBITDA_Trailing12', 0):,.0f}")
    if "SDE" in answer or "Seller" in answer:
        metrics_used.append(f"SDE: ${practice_data.data.get('Total_SDE', 0):,.0f}")
    if "Revenue" in answer:
        metrics_used.append(f"Revenue: ${practice_data.data.get('Revenue_Trailing12', 0):,.0f}")
    
    # Return complete response package
    result = {
        'question': question,
        'category': category,
        'answer': answer,
        'visualizations': viz_recommendations,
        'metrics_used': metrics_used
    }
    
    return result

# ===================================
# TEST THE Q&A SYSTEM
# ===================================

# Make sure we have practice data loaded
if 'practice_data' not in locals():
    print("⚠️ Loading practice data first...")
    practice_data = DentalPracticeData(json_path="demo_clinic.json")

# Test with different types of questions
test_questions = [
    "What is the SDE for this practice and how is it calculated?",
    "What is this practice worth based on the valuation multiples?",
    "Can this practice afford its debt payments?"
]

print("\n" + "="*60)
print("🧪 TESTING Q&A SYSTEM")
print("="*60)

# Test with the first question
test_question = test_questions[0]
result = answer_question(practice_data, test_question)

print(f"\n💡 ANSWER:\n{result['answer']}")
print(f"\n📊 Recommended Visualizations:")
for viz in result['visualizations']:
    print(f"  - {viz['title']}: {viz['description']}")
print(f"\n📈 Key Metrics Referenced:")
for metric in result['metrics_used']:
    print(f"  - {metric}")

print("\n✅ LLM interaction module ready!")


🧪 TESTING Q&A SYSTEM

📝 QUESTION: What is the SDE for this practice and how is it calculated?
📊 Category detected: SDE
🤖 Calling DeepSeek API...
✅ Response received!

💡 ANSWER:
### **Seller's Discretionary Earnings (SDE) Analysis for Demo Dental NYC**  

#### **1. Definition of SDE**  
SDE represents the total financial benefit available to a single owner-operator of the practice, including:  
- **Net profit** (pre-tax)  
- **Owner-specific addbacks** (compensation above market rate, perks, non-cash expenses, etc.)  
- **Non-recurring expenses**  

It is a key metric for small dental practice valuations, as it reflects the true cash flow available to a buyer.  

---

#### **2. SDE Calculation Breakdown**  
The SDE for **Demo Dental NYC** is **$460,000**, derived as follows:  

| **Component**                     | **Amount ($)** | **Type**       | **Justification** |
|-----------------------------------|---------------|---------------|------------------|
| Net Profit (Pre-Tax)        